link to the tutorial:  
https://raposa.trade/blog/building-a-complete-bot-trading-system-for-beginners-in-python/  
https://raposa.trade/blog/a-complete-starter-system-for-new-traders/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import timedelta

In [ ]:
def StarterSystem(data, SMA1=16, SMA2=64, target_risk=0.12, stop_loss_gap=0.5, starting_capital=1000, shorts=True):
    data['SMA1'] = data['Close'].rolling(SMA1).mean()
    data['SMA2'] = data['Close'].rolling(SMA2).mean()
    data['STD'] = data['Close'].pct_change().rolling(252).std() * np.sqrt(252)
    
    position = np.zeros(data.shape[0])
    cash = position.copy()
    stops = position.copy()
    stops[:] = np.nan
    stop_triggered = stops.copy()
    exit_on_stop_dir = 0
    for i, (ts, row) in enumerate(data.iterrows()):
        if any(np.isnan(row[["SMA1", "SMA2", "STD"]])):
            cash[i] += cash[i-1] if i > 0 else starting_capital
            continue
        
        trend_dir = 1 if row["SMA1"] > row["SMA2"] else -1
        new_stop = calcStopPrice(row["Close"], row["STD"], stop_loss_gap, trend_dir)
        # Propagate values forward.
        cash[i] = cash[i-1]
        position[i] = position[i-1]
        stops[i] = stops[i-1]
        
        if trend_dir == 1:
            if exit_on_stop_dir == -1:
                exir_on_stop_dir = 0
            if position[i] > 0:
                # Update stop.
                if new_stop > stops[i-1]:
                    stops[i] = new_stop
                
                # Check if stop was hit.
                if row["Close"] < stops[i]:
                    cash[i] += position[i] * row["Close"]
                    position[i] = 0
                    stop_triggered[i] = 1
                    exit_on_stop_dir = 1
                
                else:
                    